In [12]:
import pandas as pd
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModelForImageClassification
from torch.utils.data import Dataset
from PIL import Image
import torch

In [13]:
# choose device, not recommended to train with 'cpu

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(device)

mps


In [14]:
# obtain the models from HF

processor = AutoImageProcessor.from_pretrained("chriamue/bird-species-classifier")
model = AutoModelForImageClassification.from_pretrained("chriamue/bird-species-classifier", num_labels=200, ignore_mismatched_sizes=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/34.1M [00:00<?, ?B/s]

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at chriamue/bird-species-classifier and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([525]) in the checkpoint and torch.Size([200]) in the model instantiated
- classifier.weight: found shape torch.Size([525, 1408]) in the checkpoint and torch.Size([200, 1408]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
class BirdDataset(Dataset):
    def __init__(self, df, processor, is_test=False):
        self.df = df
        self.processor = processor
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        encoding = self.processor(image, return_tensors="pt")
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        if not self.is_test:
            encoding["labels"] = torch.tensor(int(row["label"]))
        else:
            # Solo en test devolvemos el id
            encoding["id"] = torch.tensor(int(row["id"]))

        return encoding

In [16]:
def load_data():
    # reads csvs
    train_df = pd.read_csv("../../aml-2025-feathers-in-focus/train_images.csv")
    test_df = pd.read_csv("../../aml-2025-feathers-in-focus/test_images_path.csv")

    # adjusts labels for the model
    train_df["label"] = train_df["label"] - 1

    # rewrite full image_path to have the correct folder
    train_df["image_path"] = "../../aml-2025-feathers-in-focus/train_images/train_images/" + train_df["image_path"].str.split("/").str[-1]
    test_df["image_path"] = "../../aml-2025-feathers-in-focus/test_images/test_images/" + test_df["image_path"].str.split("/").str[-1]

    # print sizes
    print(f"Train: {len(train_df)} | Test: {len(test_df)}")
    return train_df, test_df


In [17]:
train_df, test_df = load_data()

train_ds = BirdDataset(train_df, processor)
test_ds = BirdDataset(test_df, processor, is_test=True)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)


Train: 3926 | Test: 4000


In [23]:
# freeze all the parameters except the final layer
for param in model.efficientnet.parameters():  # should check if 'base_model' is all but the classifier
    param.requires_grad = False

# only the parameters of the final layer will be trained
for param in model.classifier.parameters():
    param.requires_grad = True


In [24]:
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)
epochs = 10

train_losses = []
train_accuracies = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} [TRAIN]"):
        # batch_id = batch.pop("id") 
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # accumulate loss and accuracy
        epoch_loss += loss.item() * logits.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == batch["labels"]).sum().item()
        total += logits.size(0)

    epoch_loss /= total
    epoch_acc = correct / total

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    print(f"Epoch {epoch+1} | Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")


Epoch 1 [TRAIN]:   2%|▏         | 2/123 [00:19<19:14,  9.54s/it]


KeyboardInterrupt: 

In [21]:
model.save_pretrained("saved_model_V2")
processor.save_pretrained("saved_model_V2")

['saved_model_V2/preprocessor_config.json']

In [22]:
model = AutoModelForImageClassification.from_pretrained("saved_model_V2")
processor = AutoImageProcessor.from_pretrained("saved_model_V2")

model.to(device)

EfficientNetForImageClassification(
  (efficientnet): EfficientNetModel(
    (embeddings): EfficientNetEmbeddings(
      (padding): ZeroPad2d((0, 1, 0, 1))
      (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=valid, bias=False)
      (batchnorm): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (activation): SiLU()
    )
    (encoder): EfficientNetEncoder(
      (blocks): ModuleList(
        (0): EfficientNetBlock(
          (depthwise_conv): EfficientNetDepthwiseLayer(
            (depthwise_conv_pad): ZeroPad2d((0, 1, 0, 1))
            (depthwise_conv): EfficientNetDepthwiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=32, bias=False)
            (depthwise_norm): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
            (depthwise_act): SiLU()
          )
          (squeeze_excite): EfficientNetSqueezeExciteLayer(
            (squeeze): AdaptiveAvgPool2d(output

In [ ]:
model.to(device)
model.eval()

predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        # Sacar ids y no enviarlos al modelo
        batch_ids = batch.pop("id")
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        logits = outputs.logits

        # Predicciones (1-200)
        preds = torch.argmax(logits, dim=1) + 1

        # Guardar id y label
        for i in range(len(preds)):
            predictions.append({
                "id": int(batch_ids[i].item()),  # tensor -> int
                "label": int(preds[i].item())
            })

# Crear DataFrame y guardar CSV
pred_df = pd.DataFrame(predictions)
pred_df.to_csv("submission.csv", index=False)

("Predictions saved to submission.csv")


100%|██████████| 125/125 [04:34<00:00,  2.20s/it]


'Predictions saved to submission.csv'